## Fitting a diagonal covariance Gaussian mixture model to text data

In a previous assignment, we explored k-means clustering for a high-dimensional Wikipedia dataset. We can also model this data with a mixture of Gaussians, though with increasing dimension we run into two important issues associated with using a full covariance matrix for each component.
 * Computational cost becomes prohibitive in high dimensions: score calculations have complexity cubic in the number of dimensions M if the Gaussian has a full covariance matrix.
 * A model with many parameters require more data: observe that a full covariance matrix for an M-dimensional Gaussian will have M(M+1)/2 parameters to fit. With the number of parameters growing roughly as the square of the dimension, it may quickly become impossible to find a sufficient amount of data to make good inferences.
 
En una tarea anterior, exploramos el agrupamiento de k-medias para un conjunto de datos de Wikipedia de alta dimensión. También podemos modelar estos datos con una combinación de gaussianas, aunque al aumentar la dimensión nos encontramos con dos problemas importantes asociados con el uso de una matriz de covarianza completa para cada componente.
  * El costo computacional se vuelve prohibitivo en dimensiones altas: los cálculos de puntaje tienen una complejidad cúbica en el número de dimensiones M si el Gaussiano tiene una matriz de covarianza completa.
  * Un modelo con muchos parámetros requiere más datos: observe que una matriz de covarianza completa para una Gaussiana M-dimensional tendrá M(M+1)/2 parámetros para ajustar. Con el número de parámetros creciendo aproximadamente como el cuadrado de la dimensión, rápidamente puede volverse imposible encontrar una cantidad suficiente de datos para hacer buenas inferencias.

Both of these issues are avoided if we require the covariance matrix of each component to be diagonal, as then it has only M parameters to fit and the score computation decomposes into M univariate score calculations. Recall from the lecture that the M-step for the full covariance is:

Ambos problemas se evitan si requerimos que la matriz de covarianza de cada componente sea diagonal, ya que entonces solo tiene M parámetros para ajustar y el cálculo de la puntuación se descompone en M cálculos de puntuación univariante. Recuerde de la conferencia que el paso M para la covarianza completa es:

\begin{align*}
\hat{\Sigma}_k &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_i-\hat{\mu}_k)(x_i - \hat{\mu}_k)^T
\end{align*}

Note that this is a square matrix with M rows and M columns, and the above equation implies that the (v, w) element is computed by

Tenga en cuenta que esta es una matriz cuadrada con M filas y M columnas, y la ecuación anterior implica que el elemento (v, w) se calcula mediante

\begin{align*}
\hat{\Sigma}_{k, v, w} &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})(x_{iw} - \hat{\mu}_{kw})
\end{align*}

When we assume that this is a diagonal matrix, then non-diagonal elements are assumed to be zero and we only need to compute each of the M elements along the diagonal independently using the following equation. 

Cuando suponemos que se trata de una matriz diagonal, se supone que los elementos no diagonales son cero y solo necesitamos calcular cada uno de los M elementos a lo largo de la diagonal de forma independiente mediante la siguiente ecuación.

\begin{align*}
\hat{\sigma}^2_{k, v} &= \hat{\Sigma}_{k, v, v}  \\
&= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})^2
\end{align*}

In this section, we will use an EM implementation to fit a Gaussian mixture model with **diagonal** covariances to a subset of the Wikipedia dataset. The implementation uses the above equation to compute each variance term. 

We'll begin by importing the dataset and coming up with a useful representation for each article. After running our algorithm on the data, we will explore the output to see whether we can give a meaningful interpretation to the fitted parameters in our model.

En esta sección, utilizaremos una implementación de EM para ajustar un modelo de mezcla gaussiana con covarianzas **diagonales** a un subconjunto del conjunto de datos de Wikipedia. La implementación utiliza la ecuación anterior para calcular cada término de varianza.

Comenzaremos importando el conjunto de datos y creando una representación útil para cada artículo. Después de ejecutar nuestro algoritmo en los datos, exploraremos la salida para ver si podemos dar una interpretación significativa a los parámetros ajustados en nuestro modelo.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import necessary packages

In [1]:
from __future__ import print_function # to conform python 2.x print to python 3.x
import turicreate as tc

We also have a Python file containing implementations for several functions that will be used during the course of this assignment.

In [2]:
from em_utilities import *

## Load Wikipedia data and extract TF-IDF features

Load Wikipedia data and transform each of the first 5000 document into a TF-IDF representation.

In [3]:
wiki = tc.SFrame('people_wiki.sframe/').head(5000)
wiki['tf_idf'] = tc.text_analytics.tf_idf(wiki['text'])

Using a utility we provide, we will create a sparse matrix representation of the documents. This is the same utility function you used during the previous assignment on k-means with text data.

In [4]:
wiki = wiki.add_row_number()
tf_idf, map_word_to_index = sframe_to_scipy(wiki, 'tf_idf')
map_index_to_word = dict([[map_word_to_index[i], i] for i in map_word_to_index.keys()])

As in the previous assignment, we will normalize each document's TF-IDF vector to be a unit vector. 

In [5]:
%%time
tf_idf = normalize(tf_idf)

CPU times: user 9.52 ms, sys: 1.86 ms, total: 11.4 ms
Wall time: 8.62 ms


We can check that the length (Euclidean norm) of each row is now 1.0, as expected.

In [6]:
for i in range(5):
    doc = tf_idf[i]
    print(np.linalg.norm(doc.todense()))

1.0
1.0
0.9999999999999997
1.0000000000000004
0.9999999999999999


## EM in high dimensions

EM for high-dimensional data requires some special treatment:
 * E step and M step must be vectorized as much as possible, as explicit loops are dreadfully slow in Python.
 * All operations must be cast in terms of sparse matrix operations, to take advantage of computational savings enabled by sparsity of data.
 * Initially, some words may be entirely absent from a cluster, causing the M step to produce zero mean and variance for those words.  This means any data point with one of those words will have 0 probability of being assigned to that cluster since the cluster allows for no variability (0 variance) around that count being 0 (0 mean). Since there is a small chance for those words to later appear in the cluster, we instead assign a small positive variance (~1e-10). Doing so also prevents numerical overflow.
 
EM para datos de alta dimensión requiere un tratamiento especial:
  * El paso E y el paso M deben vectorizarse tanto como sea posible, ya que los bucles explícitos son terriblemente lentos en Python.
  * Todas las operaciones deben formularse en términos de operaciones matriciales dispersas, para aprovechar los ahorros computacionales habilitados por la escasez de datos.
  * Inicialmente, algunas palabras pueden estar completamente ausentes de un grupo, lo que hace que el paso M produzca una media cero y una varianza para esas palabras. Esto significa que cualquier punto de datos con una de esas palabras tendrá una probabilidad de 0 de ser asignado a ese grupo, ya que el grupo no permite variabilidad (0 varianza) en torno a que el recuento sea 0 (0 media). Dado que existe una pequeña posibilidad de que esas palabras aparezcan más tarde en el grupo, en su lugar asignamos una pequeña variación positiva (~1e-10). Si lo hace, también evita el desbordamiento numérico.
 
We provide the complete implementation for you in the file `em_utilities.py`. For those who are interested, you can read through the code to see how the sparse matrix implementation differs from the previous assignment. 

You are expected to answer some quiz questions using the results of clustering.

Le proporcionamos la implementación completa en el archivo `em_utilities.py`. Para aquellos que estén interesados, pueden leer el código para ver cómo la implementación de la matriz dispersa difiere de la asignación anterior.

Se espera que responda algunas preguntas del cuestionario utilizando los resultados de la agrupación.

**Initializing mean parameters using k-means**

Recall from the lectures that EM for Gaussian mixtures is very sensitive to the choice of initial means. With a bad initial set of means, EM may produce clusters that span a large area and are mostly overlapping. To eliminate such bad outcomes, we first produce a suitable set of initial means by using the cluster centers from running k-means.  That is, we first run k-means and then take the final set of means from the converged solution as the initial means in our EM algorithm.

**Inicializando parámetros medios usando k-means**

Recuerde de las lecturas que EM para mezclas gaussianas es muy sensible a la elección de las medias iniciales. Con un mal conjunto inicial de medias, EM puede producir grupos que abarcan un área grande y en su mayoría se superponen. Para eliminar esos malos resultados, primero producimos un conjunto adecuado de medias iniciales utilizando los centros de clusters de la ejecución de k-medias. Es decir, primero ejecutamos k-means y luego tomamos el conjunto final de medias de la solución convergente como la media inicial en nuestro algoritmo EM.

In [13]:
help(KMeans)

Help on class KMeans in module sklearn.cluster._kmeans:

class KMeans(sklearn.base.TransformerMixin, sklearn.base.ClusterMixin, sklearn.base.BaseEstimator)
 |  KMeans(n_clusters=8, *, init='k-means++', n_init=10, max_iter=300, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='auto')
 |  
 |  K-Means clustering.
 |  
 |  Read more in the :ref:`User Guide <k_means>`.
 |  
 |  Parameters
 |  ----------
 |  
 |  n_clusters : int, default=8
 |      The number of clusters to form as well as the number of
 |      centroids to generate.
 |  
 |  init : {'k-means++', 'random'}, callable or array-like of shape             (n_clusters, n_features), default='k-means++'
 |      Method for initialization:
 |  
 |      'k-means++' : selects initial cluster centers for k-mean
 |      clustering in a smart way to speed up convergence. See section
 |      Notes in k_init for more details.
 |  
 |      'random': choose `n_clusters` observations (rows) at random from data
 |      for the i

In [15]:
%%time 

from sklearn.cluster import KMeans
from joblib import parallel_backend

np.random.seed(5)
num_clusters = 25

with parallel_backend('threading', n_jobs=1):
    # Use scikit-learn's k-means to simplify workflow
    #kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1) # uncomment to use parallelism -- may break on your installation
    kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, verbose=0)
    kmeans_model.fit(tf_idf)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    means = [centroid for centroid in centroids]

KMeans

CPU times: user 40.5 s, sys: 0 ns, total: 40.5 s
Wall time: 10.7 s


sklearn.cluster._kmeans.KMeans

**Initializing cluster weights**

We will initialize each cluster weight to be the proportion of documents assigned to that cluster by k-means above.

In [17]:
%%time 

num_docs = tf_idf.shape[0]
weights = []
for i in range(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = cluster_assignment[cluster_assignment == i].shape[0] # YOUR CODE HERE
    w = float(num_assigned) / num_docs
    weights.append(w)

CPU times: user 6.66 ms, sys: 0 ns, total: 6.66 ms
Wall time: 3.41 ms


**Initializing covariances**

To initialize our covariance parameters, we compute $\hat{\sigma}_{k, j}^2 = \sum_{i=1}^{N}(x_{i,j} - \hat{\mu}_{k, j})^2$ for each feature $j$.  For features with really tiny variances, we assign 1e-8 instead to prevent numerical instability. We do this computation in a vectorized fashion in the following code block.

In [18]:
covs = []
for i in range(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

**Running EM**

Now that we have initialized all of our parameters, run EM.

In [19]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)

In [20]:
out['loglik']

[3879297479.366981, 4883345753.533131, 4883345753.533131]

## Interpret clustering results

In contrast to k-means, EM is able to explicitly model clusters of varying sizes and proportions. The relative magnitude of variances in the word dimensions tell us much about the nature of the clusters.

A diferencia de k-means, EM puede modelar explícitamente grupos de diferentes tamaños y proporciones. La magnitud relativa de las variaciones en las dimensiones de las palabras nos dice mucho sobre la naturaleza de los clusters.

Write yourself a cluster visualizer as follows.  Examining each cluster's mean vector, list the 5 words with the largest mean values (5 most common words in the cluster). For each word, also include the associated variance parameter (diagonal element of the covariance matrix). 

Escriba usted mismo un visualizador de clúster de la siguiente manera. Al examinar el vector medio de cada grupo, enumere las 5 palabras con los valores medios más altos (las 5 palabras más comunes en el grupo). Para cada palabra, incluya también el parámetro de varianza asociado (elemento diagonal de la matriz de covarianza).

A sample output may be:
```
==========================================================
Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
football    1.08e-01    8.64e-03
season      5.80e-02    2.93e-03
club        4.48e-02    1.99e-03
league      3.94e-02    1.08e-03
played      3.83e-02    8.45e-04
...
```

In [21]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')
    
    num_clusters = len(means)
    for c in range(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(-means[c])  # YOUR CODE HERE

        for i in sorted_word_ids[:5]:
            print('{0: <12}{1:<10.2e}{2:10.2e}'.format(map_index_to_word[i], 
                                                       means[c][i],
                                                       covs[c][i]))
        print('\n==========================================================')

In [22]:
'''By EM'''
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
poetry      1.51e-01    1.90e-02
poems       6.33e-02    6.45e-03
poet        5.91e-02    6.36e-03
de          4.77e-02    8.72e-03
literary    4.68e-02    3.29e-03

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
she         1.60e-01    4.59e-03
her         1.04e-01    3.20e-03
music       1.53e-02    1.04e-03
actress     1.52e-02    1.14e-03
show        1.27e-02    7.33e-04

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
football    7.45e-02    4.57e-03
club        5.84e-02    2.55e-03
league      5.72e-02    2.83e-03
season      5.06e-02    2.35e-03
played      3.79e-02    9.46e-04

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
district    5.56e-02    4.00e-03
republican  5.47e-02    4.55e-03
senate      5.04e-02    5.21e-03
democratic  3.72e-02    2.46e-03
house       3.65e-02    2.07e

**Quiz Question**. Select all the topics that have a cluster in the model created above. [multiple choice]

**Pregunta de prueba**. Seleccione todos los temas que tienen un clúster en el modelo creado anteriormente. [opción multiple]

- Baseball -
- Basketball - 
- Soccer/Football -
- Music -
- Politics
- Law - 
- Finance

## Comparing to random initialization

Create variables for randomly initializing the EM algorithm. Complete the following code block.

In [23]:
np.random.seed(5) # See the note below to see why we set seed=5.
num_clusters = len(means)
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    # YOUR CODE HERE
    mean = np.random.normal(0, 1, num_words)
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    # YOUR CODE HERE
    cov = np.random.uniform(1,6,num_words)

    # Initially give each cluster equal weight.
    # YOUR CODE HERE
    weight = 1
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

**Quiz Question**: Try fitting EM with the random initial parameters you created above. (Use `cov_smoothing=1e-5`.) Store the result to `out_random_init`. What is the final loglikelihood that the algorithm converges to?

In [24]:
out_random_init = EM_for_high_dimension(tf_idf, random_means, random_covs, random_weights, cov_smoothing=1e-5)

In [25]:
out_random_init['loglik']

[-793165435.4353752,
 2282163613.2850647,
 2361698188.9234233,
 2362005266.1072693,
 2362023061.896716,
 2362037538.064111,
 2362037538.064111]

**R/:** 2362037538.064111

**Quiz Question:** Is the final loglikelihood larger or smaller than the final loglikelihood we obtained above when initializing EM with the results from running k-means?

In [28]:
out['loglik'][-1] < out_random_init['loglik'][-1]

False

**R/:** IS SMALLER

**Quiz Question**: For the above model, `out_random_init`, use the `visualize_EM_clusters` method you created above. Are the clusters more or less interpretable than the ones found after initializing using k-means?

**Pregunta de prueba**: para el modelo anterior, `out_random_init`, utilice el método `visualize_EM_clusters` que creó anteriormente. ¿Los grupos son más o menos interpretables que los encontrados después de inicializar usando k-means?

**R/:** MENOS INTERPRETABLE

In [27]:
# YOUR CODE HERE. Use visualize_EM_clusters, which will require you to pass in tf_idf and map_index_to_word.

visualize_EM_clusters(tf_idf, out_random_init['means'], out_random_init['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
she         2.01e-02    3.27e-03
award       1.54e-02    1.22e-03
university  1.47e-02    6.55e-04
music       1.46e-02    1.34e-03
law         1.31e-02    2.72e-03

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
she         2.08e-02    2.95e-03
league      1.92e-02    2.10e-03
football    1.60e-02    2.30e-03
season      1.55e-02    1.11e-03
her         1.42e-02    1.44e-03

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
she         5.44e-02    6.31e-03
her         4.99e-02    5.45e-03
music       1.14e-02    9.95e-04
de          1.11e-02    1.92e-03
opera       1.01e-02    3.20e-03

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
film        2.77e-02    5.42e-03
tour        2.59e-02    7.73e-03
pga         1.97e-02    7.13e-03
she         1.44e-02    1.95e-03
he          1.40e-02    1.06e

**Note**: Random initialization may sometimes produce a superior fit than k-means initialization. We do not claim that random initialization is always worse. However, this section does illustrate that random initialization often produces much worse clustering than k-means counterpart. This is the reason why we provide the particular random seed (`np.random.seed(5)`).

**Nota**: La inicialización aleatoria a veces puede producir un ajuste superior que la inicialización de k-means. No afirmamos que la inicialización aleatoria sea siempre peor. Sin embargo, esta sección ilustra que la inicialización aleatoria a menudo produce un agrupamiento mucho peor que la contraparte de k-means. Esta es la razón por la que proporcionamos la semilla aleatoria particular (`np.random.seed(5)`).

## Takeaway

In this assignment we were able to apply the EM algorithm to a mixture of Gaussians model of text data. This was made possible by modifying the model to assume a diagonal covariance for each cluster, and by modifying the implementation to use a sparse matrix representation. In the second part you explored the role of k-means initialization on the convergence of the model as well as the interpretability of the clusters.

## Llevar

En esta tarea, pudimos aplicar el algoritmo EM a una mezcla de modelos Gaussianos de datos de texto. Esto fue posible modificando el modelo para asumir una covarianza diagonal para cada grupo y modificando la implementación para usar una representación de matriz dispersa. En la segunda parte, exploró el papel de la inicialización de k-means en la convergencia del modelo, así como la interpretabilidad de los conglomerados.